In [1]:
import pandas as pd
import numpy as np

# process data1

# Load the data from the CSV file
file_path = 'listings.csv'
listings = pd.read_csv(file_path)

# Remove commas and dollar signs, then convert to numeric
# Use raw strings for regex patterns
listings['price'] = pd.to_numeric(listings['price'].replace(r'[\$,]', '', regex=True))
listings['sq_ft'] = pd.to_numeric(listings['sq_ft'].replace(r'[\,]', '', regex=True))

# Drop rows with any missing values in these columns and create a real copy
listings_cleaned = listings.dropna(subset=['price', 'sq_ft', 'year_built']).copy()

# Add a 'premium' column with initial value of 1244 for each row
listings_cleaned['premium'] = 1244

# Calculate the mean and standard deviation for the necessary columns
mean_price = listings_cleaned['price'].mean()
std_price = listings_cleaned['price'].std()
mean_sq_ft = listings_cleaned['sq_ft'].mean()
std_sq_ft = listings_cleaned['sq_ft'].std()
mean_year_built = listings_cleaned['year_built'].mean()
std_year_built = listings_cleaned['year_built'].std()

# Calculate the index for each column (how many standard deviations away from the mean)
listings_cleaned['price_index'] = (listings_cleaned['price'] - mean_price) / std_price
listings_cleaned['sq_ft_index'] = (listings_cleaned['sq_ft'] - mean_sq_ft) / std_sq_ft
listings_cleaned['year_built_index'] = (listings_cleaned['year_built'] - mean_year_built) / std_year_built

# Adjust the premium based on the indices
listings_cleaned['premium'] += (listings_cleaned['price_index'] * 100 + listings_cleaned['sq_ft_index'] * 100 + listings_cleaned['year_built_index'] * 100).astype(int)

# Sort the dataframe by 'price', 'sq_ft', and 'year_built'
listings_sorted = listings_cleaned.sort_values(by=['price', 'sq_ft', 'year_built'])

# Save the modified dataframe to a new CSV file
output_path = 'final_result.csv'
listings_sorted.to_csv(output_path, index=False)



In [2]:
import pandas as pd

#process data2 

# Load the data from a CSV file
file_path = 'latest_1000_rows.csv'
data = pd.read_csv(file_path)

# Count the frequency of each neighborhood
frequency_df = data['neighborhood_district'].value_counts().reset_index()
frequency_df.columns = ['neighborhood', 'frequency']

# Calculate the mean and standard deviation of the frequency
mean_frequency = frequency_df['frequency'].mean()
std_frequency = frequency_df['frequency'].std()

# Calculate the 'fire_index' as the number of standard deviations from the mean
frequency_df['fire_index'] = (frequency_df['frequency'] - mean_frequency) / std_frequency

# Save the DataFrame to a new CSV file
output_path = 'fire_accident.csv'
frequency_df.to_csv(output_path, index=False)

print(f"File saved as {output_path}")

File saved as fire_accident.csv


In [3]:
import pandas as pd

# add 'zipcode' features to data2 manually

# Load your CSV file
df = pd.read_csv('fire_accident.csv')

# Manually create a dictionary mapping neighborhoods to zip codes
zipcode_mapping = {
    'Tenderloin': '94102',
    'Mission': '94110',
    'South of Market': '94103, 94105, 94107',
    'Nob Hill': '94109',
    'Financial District/South Beach': '94111, 94105',
    'Bayview Hunters Point': '94124',
    'Hayes Valley': '94102, 94117',
    'Sunset/Parkside': '94116, 94122',
    'Western Addition': '94115, 94117',
    'Chinatown': '94108',
    'Pacific Heights': '94115, 94123',
    'Marina': '94123',
    'Russian Hill': '94109, 94133',
    'Excelsior': '94112',
    'Inner Sunset': '94122',
    'Noe Valley': '94114',
    'Outer Richmond': '94121',
    'Mission Bay': '94158',
    'Potrero Hill': '94107',
    'Castro/Upper Market': '94114',
    'Outer Mission': '94112',
    'Lone Mountain/USF': '94117, 94118',
    'Bernal Heights': '94110',
    'Haight Ashbury': '94117',
    'West of Twin Peaks': '94127, 94131',
    'Inner Richmond': '94118',
    'Presidio Heights': '94118',
    'Lakeshore': '94132',
    'North Beach': '94133',
    'Oceanview/Merced/Ingleside': '94112, 94132',
    'Portola': '94134',
    'Glen Park': '94131',
    'Presidio': '94129',
    'Japantown': '94115',
    'Twin Peaks': '94131',
    'Golden Gate Park': '94122',
    'Visitacion Valley': '94134',
    'Seacliff': '94121',
    'Treasure Island': '94130',
    'McLaren Park': '94134',
    'Lincoln Park': '94121'
}

# Assuming your neighborhoods column is named correctly in your CSV. Replace 'Neighborhood' with the correct column name if different.
df['zipcode'] = df['neighborhood'].map(zipcode_mapping)

# Save the DataFrame back to the same CSV file
df.to_csv('fire_accident.csv', index=False)

In [4]:
import pandas as pd

#add 'zipcode' features to data1, which is the last 5 digits of cells in column 'address'

# Load the CSV file
file_path = 'final_result.csv'
data = pd.read_csv(file_path)

# Extract the last 5 characters from the 'address' column and convert them to an integer
data['zipcode'] = data['address'].str[-5:].astype(int)

# Save the updated DataFrame to a new CSV file
output_file_path = 'final_result.csv'
data.to_csv(output_file_path, index=False)


In [5]:
import pandas as pd

#connect data2 to data1

# Load the data
final_result_df = pd.read_csv('final_result.csv')
fire_accident_df = pd.read_csv('fire_accident.csv')

# Expand rows with multiple zipcodes in 'fire_accident.csv'
expanded_fire_accident = fire_accident_df.set_index(['neighborhood', 'frequency', 'fire_index'])['zipcode'] \
                                          .str.split(', ', expand=True) \
                                          .stack() \
                                          .reset_index(name='zipcode') \
                                          .drop('level_3', axis=1)

# Convert 'zipcode' to string in both dataframes to ensure matching data types
final_result_df['zipcode'] = final_result_df['zipcode'].astype(str)
expanded_fire_accident['zipcode'] = expanded_fire_accident['zipcode'].astype(str)

# Merge dataframes and sort by 'fire_index' to prepare for picking the maximum per original row
final_result_remerged = pd.merge(final_result_df.reset_index(), expanded_fire_accident, on='zipcode', how='left')
final_result_sorted = final_result_remerged.sort_values(by=['index', 'fire_index'], ascending=[True, False])

# Drop duplicates based on the 'index' to keep only the row with the highest 'fire_index' for each original entry
final_result_deduplicated = final_result_sorted.drop_duplicates(subset='index', keep='first')

# Reset index for the cleaned DataFrame and replace NaNs in 'fire_index' with 0
final_result_final = final_result_deduplicated.sort_values(by='index').reset_index(drop=True)
final_result_final['fire_index'] = final_result_final['fire_index'].fillna(0)

# Ensure the DataFrame retains only the relevant columns from the original DataFrame
final_columns = final_result_df.columns.tolist() + ['neighborhood', 'fire_index']
final_result_final = final_result_final[final_columns]

# Overwrite the existing 'final_result.csv' file
final_result_final.to_csv('final_result.csv', index=False)

In [6]:
import pandas as pd

#make changes to premium after adding fire_index from data2

# Load the data from the CSV file
data_path = 'final_result.csv'
data = pd.read_csv(data_path)

# Update the 'premium' values: multiply 'fire_index' by 50 and add it to 'premium'
data['premium'] = data['fire_index'] * 50 + data['premium']

# Overwrite the original CSV file with the modified data
data.to_csv(data_path, index=False)

In [7]:
import pandas as pd

# Load the original crime data
data_path = 'san_francisco_crime_data_by_neighborhood.csv'
crime_data = pd.read_csv(data_path)

# Define categories for high, medium, and low risk crimes
high_risk = ['Homicide', 'Robbery', 'Arson', 'Assault', 'Sex Offense', 'Weapons Offense', 'Kidnapping']
medium_risk = [
    'Burglary', 'Drug Offense', 'Drug Violation', 'Embezzlement', 'Forgery And Counterfeiting', 'Fraud',
    'Motor Vehicle Theft', 'Prostitution', 'Stolen Property', 'Weapons Carrying Etc', 'Vandalism',
    'Traffic Violation Arrest', 'Larceny Theft'
]
low_risk = [
    'Case Closure', 'Courtesy Report', 'Disorderly Conduct', 'Fire Report', 'Liquor Laws', 'Lost Property',
    'Malicious Mischief', 'Miscellaneous Investigation', 'Missing Person', 'Non-Criminal',
    'Offences Against The Family And Children', 'Other', 'Other Miscellaneous', 'Other Offenses',
    'Recovered Vehicle', 'Suicide', 'Suspicious Occ', 'Traffic Collision', 'Vehicle Impounded', 'Warrant'
]

# Group data by 'Location' and count occurrences in each crime category
grouped = crime_data.groupby('Location')
high_count = grouped['Category'].apply(lambda x: x.isin(high_risk).sum()).rename('high_risk_frequency')
medium_count = grouped['Category'].apply(lambda x: x.isin(medium_risk).sum()).rename('medium_risk_frequency')
low_count = grouped['Category'].apply(lambda x: x.isin(low_risk).sum()).rename('low_risk_frequency')

# Combine these counts into a single DataFrame
summary = pd.concat([high_count, medium_count, low_count], axis=1).reset_index()
summary.rename(columns={'Location': 'neighborhood_c'}, inplace=True)

# Calculate mean and standard deviation for the risk frequencies
means = summary[['high_risk_frequency', 'medium_risk_frequency', 'low_risk_frequency']].mean()
stds = summary[['high_risk_frequency', 'medium_risk_frequency', 'low_risk_frequency']].std()

# Calculate indices as the number of standard deviations from the mean
for column in ['high_risk_frequency', 'medium_risk_frequency', 'low_risk_frequency']:
    index_column = column.split('_')[0] + '_risk_index'
    summary[index_column] = ((summary[column] - means[column]) / stds[column]).round(2)

# Save the DataFrame to a CSV file named 'summary.csv'
summary.to_csv('crime_summary.csv', index=False)



In [8]:
import pandas as pd

# Load the data from the provided CSV files
final_result = pd.read_csv('final_result.csv')
crime_summary = pd.read_csv('crime_summary.csv')

# Merge 'crime_summary' data into 'final_result' using the 'neighborhood' and 'neighborhood_c' columns
final_result = pd.merge(
    final_result,
    crime_summary[['neighborhood_c', 'high_risk_index', 'medium_risk_index', 'low_risk_index']],
    left_on='neighborhood',
    right_on='neighborhood_c',
    how='left'
)

# Drop the 'neighborhood_c' column from 'final_result' after merging
final_result.drop(columns=['neighborhood_c'], inplace=True)

# Incrementally update the 'premium' column based on the risk indices
# Assume initial 'premium' is already set in the dataset
final_result['premium'] += (
    final_result['high_risk_index'] * 50 +
    final_result['medium_risk_index'] * 30 +
    final_result['low_risk_index'] * 10
)

# Overwrite the original 'final_result.csv' with the updated data
final_result.to_csv('final_result.csv', index=False)







In [9]:
import pandas as pd

#Final Adjustments

# Load the data
data = pd.read_csv('final_result.csv')

# Change the first letter of each column name to uppercase
data.columns = [column.capitalize() for column in data.columns]

# Rearrange columns to make 'Premium' the first column
column_order = ['Premium'] + [col for col in data.columns if col != 'Premium']
data = data[column_order]

# Format 'Premium' and 'Price' columns to dollar format
data['Premium'] = data['Premium'].apply(lambda x: f"${x:,.2f}")
data['Price'] = data['Price'].apply(lambda x: f"${x:,.2f}")

# Overwrite the original CSV file with the modified data
data.to_csv('final_result.csv', index=False)
